In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image

%matplotlib inline

In [4]:
def yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = .6):
    """ filters yolo boxes by thresholding on object and class confidence """
    
    #Compute box scores
    box_scores = box_confidence*box_class_probs

    #Find the box_classes
    box_classes = tf.math.argmax(box_scores,axis=-1)
    box_class_scores = tf.math.reduce_max(box_scores,axis=-1)
    
    #Create a filtering mask
    filtering_mask = box_class_scores >= threshold
    
    #Apply the mask
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes = tf.boolean_mask(box_classes,filtering_mask)
    
    return scores, boxes, classes

In [24]:
def iou(box1, box2):
    """implement the intersection over union between boxes"""

    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    # Calculate intersection of box1 and box2
    xi1 = max(box1_x1,box2_x1)
    yi1 = max(box1_y1,box2_y1)
    xi2 = min(box1_x2,box2_x2)
    yi2 = min(box1_y2,box2_y2)
    inter_width = xi2 - xi1
    inter_height =  yi2 - yi1
    inter_area = max(inter_height, 0)*max(inter_width, 0)
    
    # compute the IoU
    box1_area = (box1_x2 - box1_x1)*(box1_y2-box1_y1)
    box2_area = (box2_x2 - box2_x1)*(box2_y2-box2_y1)
    union_area = box1_area + box2_area - inter_area
    iou = inter_area/union_area
    
    return iou

In [30]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    
    """ applies non max suppression """

    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32')

    #get the list of boxes indices
    nms_indices = tf.image.non_max_suppression(boxes,scores,max_boxes_tensor,iou_threshold=0.5,name=None)
    
    #select only nms_indices from scores, boxes and classes
    scores = tf.gather(scores,nms_indices)
    boxes = tf.gather(boxes,nms_indices)
    classes = tf.gather(classes,nms_indices)

    return scores, boxes, classes

In [41]:
def yolo_boxes_to_corners(box_xy, box_wh):
    """ convert yolo box to bounding box """
    box_min  = box_xy - (box_wh / 2.)
    box_maxe  = box_xy + (box_wh / 2.)

    return tf.keras.backend.concatenate([
        box_min [..., 1:2],  # y_min
        box_min [..., 0:1],  # x_min
        box_maxe [..., 1:2],  # y_max
        box_maxe [..., 0:1]  # x_max
    ])


In [42]:
def yolo_eval(yolo_outputs, image_shape = (720, 1280), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    
    """ converts the output of yolo to predicted boxes """

    # Retrieve outputs of the YOLO model
    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs
    
    # Convert boxes
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    
    #score filtering
    scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = score_threshold)
    # scale back to original image shape 
    boxes = scale_boxes(boxes, image_shape)
    
    # Non-max suppression
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes = max_boxes, iou_threshold = iou_threshold)
    
    return scores, boxes, classes

In [44]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
model_image_size = (608, 608) # Same as yolo_model input layer size

In [45]:
yolo_model = load_model("model_data/", compile=False)

In [46]:
yolo_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 608, 608, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 608, 608, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 608, 608, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
def yolo_head(feats, anchors, num_classes):
    """convert final layer to bounding box """

    num_anchors = len(anchors)
    #reshape to batch, height, width, num_anchors, box_params 
    anchors_tensor = K.reshape(K.variable(anchors), [1, 1, 1, num_anchors, 2])

    conv_dims = K.shape(feats)[1:3]  # assuming channels last

    conv_height_index = K.arange(0, stop=conv_dims[0])
    conv_width_index = K.arange(0, stop=conv_dims[1])
    conv_height_index = K.tile(conv_height_index, [conv_dims[1]])

    conv_width_index = K.tile(
        K.expand_dims(conv_width_index, 0), [conv_dims[0], 1])
    conv_width_index = K.flatten(K.transpose(conv_width_index))
    conv_index = K.transpose(K.stack([conv_height_index, conv_width_index]))
    conv_index = K.reshape(conv_index, [1, conv_dims[0], conv_dims[1], 1, 2])
    conv_index = K.cast(conv_index, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, conv_dims[0], conv_dims[1], num_anchors, num_classes + 5])
    conv_dims = K.cast(K.reshape(conv_dims, [1, 1, 1, 1, 2]), K.dtype(feats))

    box_xy = K.sigmoid(feats[..., :2])
    box_wh = K.exp(feats[..., 2:4])
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.softmax(feats[..., 5:])

    box_xy = (box_xy + conv_index) / conv_dims
    box_wh = box_wh * anchors_tensor / conv_dims

    return box_xy, box_wh, box_confidence, box_class_probs

In [47]:
def predict(image_file):
    # preprocessing
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
    
    yolo_model_outputs = yolo_model(image_data)
    yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))
    
    out_scores, out_boxes, out_classes = yolo_eval(yolo_outputs, [image.size[1],  image.size[0]], 10, 0.3, 0.5)

    # predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), "images/" + image_file))
    # generate colors for boxes.
    colors = get_colors_for_classes(len(class_names))
    # draw bounding boxes on image
    draw_boxes(image, out_boxes, out_classes, class_names, out_scores)
    # save the predicted bounding box on image
    image.save(os.path.join("out", image_file), quality=100)
    # Display the results
    output_image = Image.open(os.path.join("out", image_file))
    imshow(output_image)

    return out_scores, out_boxes, out_classes

In [48]:
out_scores, out_boxes, out_classes = predict("test.jpg")

ValueError: Shapes (1, 19, 19, 1) and (1, 19, 19, 5) are incompatible